# Solving integer partition problems on a D-Wave system

**Authored by David Radcliffe (dradcliffe@gmail.com) on March 13, 2023.**


The **integer partition problem** is as follows: 

> Given an list of integers, split the list into two parts having equal sums. 

For example, the list [15, 20,  8,  5,  9,  9]
can be split into two parts, [15, 9, 9] and [20, 8, 5]
each summing to 33.

This problem is NP-complete, so there is no known efficient algorithm that can solve it in general. However, if the integers are not too large, the problem can be solved efficiently using a dynamic programming, and there also exist efficient approximate algorithms.

A typical application is scheduling a set of independent tasks between two processors so that the total processing time is minimized. Considering our previous example, suppose that we have six tasks, requiring 15, 20, 8, 5, 9, and 9 minutes, respectively. Then we could assign the first task and the last two tasks to the first processor, and the remaining three tasks to the second processor. The first processor completes its tasks in 15 + 9 + 9 = 33 minutes, and the second processor completes its tasks in 20 + 8 + 5 = 33 minutes, so this is an optimal allocation of tasks between the two processors.

In this notebook, we attempt to solve integer partition problems on a D-Wave system, using a hybrid (quantum and classical) algorithm. The D-Wave system is not a universal (gate-based) quantum computer, but rather a quantum annealer. It is unknown whether such devices are able to solve combinatorial problems more efficently than a classical computer.

**Note:** You will need to install the [dwave-system](https://docs.ocean.dwavesys.com/projects/system/en/latest/index.html) package and obtain an [API token](https://docs.ocean.dwavesys.com/en/stable/overview/sapi.html) to run this notebook. You can enter your API token by executing the cell below. For security reasons, you should not save your API token in this notebook.


In [1]:
from getpass import getpass
api_token = getpass('Enter API token for D-Wave: ')

Enter API token for D-Wave: ········


In [10]:
# Execute this cell unless you have already installed the dwave-system package.
!pip install dwave-system -q

In [3]:
import numpy as np
from dwave.system import LeapHybridSampler

In [4]:
"""
Create an instance of the integer partition problem.

Args:
size (int): The total size of the partition instance.
max_value (int): The maximum value that can occur as an element in the returned array.

Returns:
numpy.ndarray: An array of size 'size', where (size // 2) of the elements have the same sum as the 
remaining (size - size // 2) elements. The values in the array are randomly generated integers 
between 1 and max_value inclusive.

Raises:
AssertionError: If the generated array does not meet the partition criteria.

Note that this function uses (up to) 64-bit signed integers, and overflow errors may occur
if the parameters are too large.

Example:
>>> create_partition_instance(6, 100)
array([51, 99, 58, 80, 69, 95])

This is a valid instance because 51 + 80 + 95 = 99 + 58 + 69.
"""
def create_partition_instance(size, max_value):
    while True:
        n = size // 2
        m = size - n
        a = np.random.randint(1, max_value + 1, size = n)
        b = np.random.randint(1, max_value + 1, size = m)
        diff = b.sum() - a.sum()
        r = diff % n
        a += diff // n
        a[:r] += 1
        assert a.sum() == b.sum()
        a = np.concatenate([a, b])
        if a.min() >= 1 and a.max() <= max_value:
            np.random.shuffle(a)
            return a

The first step is to formulate our problem as a quadratic optimization problem.

Let $\mathbf{s} = (s_1, \ldots, s_n)^{\top}$ be a column vector of integers. A partition of $\mathbf{s}$ into two subvectors is defined by a vector $\mathbf{x} = (x_1, \ldots, x_n)^{\top}$ where $x_k = \pm1$ for all $k$.
The entry $s_k$ is assigned to the left part when $x_k = -1$, and it is assigned to the right part
when $x_k = 1$.

The inner product $\mathbf{x} \cdot \mathbf{s}^{\top}$ is the (signed) difference between the sums of the two parts. It suffices to minimize the square of this quantity. Note that

$$(\mathbf{x} \cdot \mathbf{s}^{\top})^2 = 
   \mathbf{x}^{\top} \mathbf{s}\mathbf{s}^{\top} \mathbf{x} =
   \mathbf{x}^{\top}J\mathbf{x},$$
where $J = \mathbf{s} \mathbf{s}^{\top}$. 

Thus, our problem can be formulated as a quadratic optimization problem with no constraints,
except that the entries of the solution vector are $\pm 1$. This type of optimization problem is known
as the **Ising model**.

In [5]:
"""
Solves an instance of the integer partition problem on the D-Wave system using an Ising model.

Given an instance of the integer partition problem represented as an array of integers, this function
constructs an Ising model that encodes the problem and solves it on the D-Wave quantum annealer using the
LeapHybridSampler. The resulting solution is then returned as an array of signed ones representing the
partition of the input array, where a value of -1 at index i indicates that the ith element of the input array
belongs to the left partition, and a value of +1 indicates that the element belongs to the right partition.

Args:
instance (numpy.ndarray): The input array to be partitioned.

Returns:
numpy.ndarray: An array of signed ones representing the partition of the input array, where a value
of -1 at index i indicates that the ith element of the input array belongs to the left partition, and
a value of +1 indicates that the element belongs to the right partition.

Example:
>>> instance = np.array([1, 7, 11, 5])
>>> solve_partition_ising(instance)
array([-1, 1, -1, 1])
"""
def solve_partition_ising(instance):
    J = matrix_to_dict(np.outer(instance, instance))
    sample = LeapHybridSampler(token=api_token).sample_ising({}, J).samples()[0]
    solution = np.array(list(sample.values()))
    return solution

In [6]:
"""
Converts a given numpy matrix into a dictionary.

Given a numpy matrix, this function converts the matrix into a dictionary representation where the keys
are tuples (i, j) corresponding to the indices of the matrix, and the values are the elements of the matrix
at those indices.

Args:
matrix (numpy.ndarray): The input matrix to be converted.

Returns:
dict: A dictionary representation of the input matrix.

Example:
>>> matrix = np.array([[1, 2], [3, 4]])
>>> matrix_to_dict(matrix)
{(0, 0): 1, (0, 1): 2, (1, 0): 3, (1, 1): 4}
"""
def matrix_to_dict(matrix):
    return dict(np.ndenumerate(matrix))

Let's create a small instance of the partition problem. We will generate an array of 50 integers, between 1 and 65535 inclusive, that can be partitioned into two subarrays with equal sums.

In [7]:
instance = create_partition_instance(50, 2**16 - 1)
instance

array([57090, 44233, 13457,  4691, 60476, 60088,  8793, 45039, 10350,
       10887, 52122, 11195, 59079, 54718, 50233,  1282, 29165, 39442,
       26798, 58811,  1219, 15640,  3622, 26764, 62373, 44830, 32882,
       12109, 58279, 49762, 44658, 36927, 17248, 52622, 46183,  4168,
       57719, 39421, 24881, 36063, 53599, 14416,  8595, 33404, 25049,
       44726,  1386, 24170, 20787, 39139])

Next, we use our Ising solver to partition the instance. This might take a minute, so be patient!

In [8]:
solution = solve_partition_ising(instance)
solution

array([-1, -1,  1, -1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,
       -1, -1,  1,  1, -1,  1, -1,  1, -1,  1, -1, -1, -1,  1,  1, -1, -1,
        1,  1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,  1,  1,  1,  1],
      dtype=int8)

We can use the dot product to measure the quality of our solution. The dot product of the instance and solution vectors is equal to the sum of the right partition minus the sum of the left partition. If the dot product is zero then the solution is perfect; otherwise it is only approximate.

In [9]:
solution.dot(instance)

-178